In [1]:
import requests
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import json
from flatten_json import flatten

# Plotting settings
import mpld3
mpld3.enable_notebook()
%matplotlib notebook
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,5)

In [2]:
class farm_data():
    
    def __init__(self, sensor_id, start_date, end_date, max_items = '1000', auth_token = None):
        
        self.dev_id = sensor_id
        self.device_uid = 'device_uid='+self.dev_id+'&'
        self.date = 'start_date='+start_date+'&'+'end_date='+end_date
        self.limit = '&limit='+max_items
        self.base_url = 'http://pwrnet-158117.appspot.com/api/v1/farm_device_data/?'
        self.auth_token = 'adde8a3993d902769197c1db98a9ab0e88dce587'
        self.headers = {'Authorization': 'Token ' + self.auth_token}
        
    def get_data(self, url=None):
        # Check to see if id is valid
        # The first case should not be none - it should be the url -> maybe remove the url=None
        if url is None:
            try:
                resp = requests.get(self.base_url+self.device_uid+self.date+self.limit, headers=self.headers)
                resp.raise_for_status()
            except requests.exceptions.HTTPError as err:
                return None
        else:
            try:
                resp = requests.get(url, headers=self.headers)
                resp.raise_for_status()
            except requests.exceptions.HTTPError as err:
                return None
        
        data = resp.json()['results']
        if resp.json()['next'] is None:
            return data
        else:
            return (data+self.get_data(url=resp.json()['next']))
            
            
    def convert2pandas(self, raw_data):
        dev_id = int(self.dev_id)
        if raw_data is None:
            print('No data collected')
            return None
        # Checking for Lora sensors - need to automate this in case the number of lora devices grow (pull info from FarmDevice endpoint or check what type of sensr the id corresponds to)
        if dev_id < 24 and dev_id > 0:
            data_flat = [flatten(json.loads(d['device_data'])) for d in raw_data]
            return pd.DataFrame(data_flat)
        # Checking for egauge
        elif dev_id == 46613:
            data_flat = [flatten(json.loads(d['device_data'])) for d in raw_data]
            df = pd.DataFrame(data_flat)
            return df[df.columns[~df.columns.str.contains('raw_')]] # returning only processed data
        # Checking for battery
        elif dev_id == 67682 or dev_id == 67670:
            data_flat = [flatten(d['device_data']) for d in raw_data]
            return pd.DataFrame(data_flat) 
        # Checking for STD
        elif dev_id == 100000:
            df_list = []
            for d in raw_data:
                df_list.append(pd.DataFrame(d['device_data']))
            return pd.concat(df_list)
        else:
            print('No parsing available for this device ID')
            return None


In [3]:
# Getting data from STD
std_object = farm_data('100000','2019-08-28', '2019-10-31', max_items = '1000')
std_data = std_object.get_data()
std_pd = std_object.convert2pandas(std_data)